In [1]:
import os
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
client = Client()

In [3]:
NYC_TAXI_DATA_CSV = '/home/spotter/Documents/job/coiled_assessment/data/nyc-taxi/part-*.csv'

# Task 2 Load & Clean Data; Save to Parquet

In [4]:
df = dd.read_csv(NYC_TAXI_DATA_CSV, dtype={'store_and_fwd_flag': str})
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,N,151,239,1.0,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1.0,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,N,239,246,1.0,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2.0,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,N,236,236,1.0,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2.0,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2.0,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,N,193,193,2.0,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


## Basic Filters

In [5]:
# Clearly negative values for fairs and tips are unreasonable
# Setting an intuitive guess that a 10k USD cap on total fair will suffice
mask = (df['tip_amount'] > 0) & (df['total_amount'] > 0) & (df['total_amount'] < 10000)
filtered = df[mask]

In [6]:
filtered[['total_amount', 'tip_amount']].describe().compute()

,total_amount,tip_amount
count,5.823267e+07,5.823267e+07
mean,2.015605e+01,3.179025e+00
std,1.525548e+01,2.885673e+00
min,1.100000e-01,1.000000e-02
25%,1.266000e+01,1.950000e+00
50%,1.656000e+01,2.660000e+00
75%,8.319750e+01,1.217250e+01
max,1.199160e+03,7.872500e+02


## Save to Parquet format

In [7]:
NYC_TAXI_DATA_PARQUET = '/home/spotter/Documents/job/coiled_assessment/data/parquet/nyc-taxi'
if not os.path.exists(NYC_TAXI_DATA_PARQUET):
    filtered.to_parquet(NYC_TAXI_DATA_PARQUET, schema='infer')

# Task 3 Performance Considerations for Task 2

Some thoughts
- How is the new data being loaded. Are we really just adding daily csvs?
- Can wo do filtering or do we need to store raw data
- How important is consistency of new data?
- Is parquet able to be "appended" to instead of just created in new each day?